In [1]:
import pandas as pd
from datetime import datetime
import os

# Load latest DC Stock
def get_latest_Stock_DC_file(directory, file_prefix, file_extension):
    # Match the prefix and extension
    masterfiles = [f for f in os.listdir(directory) if f.startswith(file_prefix) and f.endswith(file_extension)]

    # Extract date from filenames
    dates = []
    for file in masterfiles:
        try:
            date_str = file.replace(file_prefix, '').replace(file_extension, '').strip()
            date_obj = datetime.strptime(date_str, '%d-%m-%Y')
            dates.append((file, date_obj))
        except ValueError:
            # If the filename doesn't match then skip
            continue

    # Sort files by name and return the latest file
    if dates:
        latest_file = max(dates, key=lambda x: x[1])[0]
        return os.path.join(directory, latest_file)
    else:
        return None


# Clean DC file Stock
def clean_DC_file(source_directory, destination_directory):
    file_prefix = 'DC_End'
    file_extension = '.xlsx'
    latest_file_path = get_latest_Stock_DC_file(source_directory, file_prefix, file_extension)  # Step to find the latest file

    if latest_file_path:
        print(f"Loading the latest DC Stock File: {latest_file_path}")
        master_df = pd.read_excel(latest_file_path, sheet_name='Sheet1')

        master_df.rename(columns={
            master_df.columns[2]: 'DC_Name', 
            master_df.columns[4]: 'Product Name'}, inplace=True)

        # Change the data in columns plant 
        master_df['Plant'] = master_df['Plant'].map({
            'D001': 'DC1',
            'D002': 'DC2',
            'D004': 'DC4'
        })

        # Replace <0 with 0
        master_df['Stock Qty'] = master_df['Stock Qty'].where(master_df['Stock Qty'] >= 0, 0)
        master_df['Stock Value'] = master_df['Stock Value'].where(master_df['Stock Value'] >= 0, 0)
        print("Replace Stock Qty <0 to '0'")

        # Filter rows where Stock Qty is greater than 0 for pivot
        master_df_filtered = master_df[master_df['Stock Qty'] > 0]
        print("Pivoting filtered data Stock >0")

        # Pivot data
        pivoted_df = master_df_filtered.pivot_table(
            index=['Material'],
            columns='Plant',
            values=['Stock Qty', 'Stock Value'],
            aggfunc='sum',
            fill_value=0
        ).reset_index()
    

        # Flatten the multi-level columns and retain only the specified suffixes
        pivoted_df.rename(columns={'Material': 'CJ_Item'},inplace=True)
        pivoted_df.columns = [
            f"{plant}_Remain_StockQty" if value == 'Stock Qty' else f"{plant}_Remain_StockValue"
            if plant else 'CJ_Item'  # Keep 'CJ_Item' column unchanged
            for value, plant in pivoted_df.columns
        ]

        exclude_cj_path = r'D:\Data for Stock Report\Exclude CJ_Item.xlsx'
        exclude_df = pd.read_excel(exclude_cj_path, sheet_name='Exclude')
        print("Loading product list from Company Code: 101318 and 401155 to exclude from this file")

        pivoted_df = pivoted_df.merge(exclude_df,on='CJ_Item',how='left',indicator='_from-merged')
        
        pivoted_df = pivoted_df[pivoted_df['_from-merged'] !='both']
        pivoted_df.drop('_from-merged',axis=1,inplace=True)

        # Save the cleaned data as an .xlsx file
        cleaned_file_path = os.path.join(destination_directory, 'cleaned_DC_daily_stock.xlsx')
        with pd.ExcelWriter(cleaned_file_path, mode='w') as writer:
            pivoted_df.to_excel(writer, sheet_name='Pivot_DC_stock', index=False)
            master_df.to_excel(writer, sheet_name='cleaned data', index=False)
        print(f"Daily DC Stock File has been cleaned & saved to: {cleaned_file_path}")


# Main function
def main():
    # Define directories
    source_directory = r'T:\SCM Data\Data For Stock\DC_DailyStock'
    destination_directory = r'D:\Data for Stock Report'

    # Clean the latest master file and save to the destination
    clean_DC_file(source_directory, destination_directory)


if __name__ == "__main__":
    main()


Loading the latest DC Stock File: T:\SCM Data\Data For Stock\DC_DailyStock\DC_End02-01-2025.xlsx
Replace Stock Qty <0 to '0'
Pivoting filtered data Stock >0
Loading product list from Company Code: 101318 and 401155 to exclude from this file
Daily DC Stock File has been cleaned & saved to: D:\Data for Stock Report\cleaned_DC_daily_stock.xlsx
